In [1]:
#Importing libraries
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from matplotlib.animation import FuncAnimation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from keras.layers import LeakyReLU

In [2]:
# Defining necessary parameters
epochs = 1 # 600
batch_size = 512
plot_freq = 1
latent_dim = 1

## Define a Discriminator Model

In [3]:
# Build a discriminator neural network
def build_discriminator(dim):
  model = Sequential()
  for _ in range(2):
    model.add(Dense(64,input_dim=dim,activation=LeakyReLU(alpha=0.1)))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(Adam(learning_rate=0.002, beta_1=0.5),loss='binary_crossentropy',metrics=['accuracy'])
  return model

## Define a Generator Model

In [4]:
# Build a generator neural network
def build_generator(latent_dim, output_dim):
  model = Sequential()
  for _ in range(4):
    model.add(Dense(16,input_dim=latent_dim,activation=LeakyReLU(alpha=0.1)))
  model.add(Dense(output_dim))

  return model

## Defining and Training the GAN


In [5]:
# Given a generator and a discriminator, build a GAN
def build_GAN(G, D, latent_dim):
  D.trainable = False
  input_layer = tf.keras.layers.Input((latent_dim,))
  X = G(input_layer)
  output_layer = D(X)
  GAN = Model(input_layer, output_layer)
  GAN.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return GAN

In [6]:
# Generate random uniform noise to input to the generator
def generate_input_noise(batch_size, latent_dim):
    return np.random.rand(batch_size,latent_dim)

In [7]:
#  Generate real data from a normal distribution to train discriminator
def get_real_data(n_samples,output_dim):
  np.random.seed(109)
  return np.random.randn(n_samples, output_dim)

In [8]:
# Build the GAN
G = build_generator(latent_dim, 1)
D = build_discriminator(1)
GAN = build_GAN(G, D, latent_dim)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Training the GAN
D_loss = []
G_loss = []
G_predict=[]

for step in range(epochs):

    np.random.seed(109+step)

    # Train discriminator
    real_data = get_real_data(batch_size // 2, 1)
    fake_data = G.predict(generate_input_noise(batch_size // 2, latent_dim), batch_size=batch_size // 2)
    data = np.concatenate((real_data, fake_data), axis=0)

    labels = np.concatenate((np.ones((batch_size // 2, 1)), np.zeros((batch_size // 2, 1))), axis=0)
    _D_loss, _ = D.train_on_batch(data, labels)

    # Train generator
    # While generator training we do not want discriminator weights to be adjusted.
    D.trainable = False
    noise = generate_input_noise(batch_size, latent_dim)
    labels = np.ones((batch_size, 1))
    print(GAN.train_on_batch(noise, labels))
    _G_loss = GAN.train_on_batch(noise, labels)

    D_loss.append(_D_loss)
    G_loss.append(_G_loss)


    test_noise = generate_input_noise(10000, latent_dim)
    fake_samples = G.predict(test_noise, batch_size=len(test_noise))
    G_predict.append(fake_samples)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step


[array(0.6705596, dtype=float32), array(0.6705596, dtype=float32), array(1., dtype=float32), array(1., dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step


In [12]:
disc_loss=_D_loss
gen_loss=_G_loss

In [14]:
fig, ax = plt.subplots(1, 2, figsize=(20,8))
plt.close(fig)
def animate(i):
  ax[1].cla()
  # Plot loss and accuracy
  ax[0].plot(np.arange(4*i), G_loss[0:4*i],label='G loss',c='darkred',zorder=50,alpha=0.8)
  ax[0].plot(np.arange(4*i), D_loss[0:4*i],label='D loss',c='darkblue',zorder=55,alpha=0.8)
  ax[0].set_xlim(-5, epochs+5)
  ax[0].set_ylim(-0.05, 1.55)
  ax[0].set_xlabel('Epoch')

  #Plot distributions
  x_vals = np.linspace(-3, 3, 301)
  y_vals = stats.norm(0,1).pdf(x_vals)
  ax[1].plot(x_vals, y_vals, color='blue', label='real')
  ax[1].fill_between(x_vals, np.zeros(len(x_vals)), y_vals, color='blue', alpha=0.6)
  a = sns.kdeplot(G_predict[4*i].flatten(), color='red', alpha=0.6, label='GAN', ax=ax[1], shade=True)
  ax[1].set_xlim(-3, 3)
  ax[1].set_ylim(0, 0.82)
  ax[1].set_xlabel('Sample Space')
  ax[1].set_ylabel('Probability Density')

simulation = FuncAnimation(fig, animate, frames=epochs//4, interval=100, repeat=True)
HTML(simulation.to_html5_video())

/usr/local/lib/python3.10/dist-packages/matplotlib/animation.py:1740: UserWarning: Can not start iterating the frames for the initial draw. This can be caused by passing in a 0 length sequence for *frames*.

If you passed *frames* as a generator it may be exhausted due to a previous display or save.
  warnings.warn(
